In [3]:
cd scripts

F:\tensorflow\scripts


C:\Users\srija\anaconda3\envs\tf\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
# Create train data:
!python generate_tfrecord.py -x F:\tensorflow\training_demo\images\test -l F:\tensorflow\training_demo/annotations/label_map.pbtxt -o F:\tensorflow\training_demo/annotations/train.record

# Create test data:
!python generate_tfrecord.py -x F:\tensorflow\training_demo\images\train -l F:\tensorflow\training_demo/annotations/label_map.pbtxt -o F:\tensorflow\training_demo/annotations/test.record

Successfully created the TFRecord file: F:\tensorflow\training_demo/annotations/train.record
Successfully created the TFRecord file: F:\tensorflow\training_demo/annotations/test.record


In [1]:
cd F:\tensorflow\training_demo

F:\tensorflow\training_demo


C:\Users\srija\anaconda3\envs\tf\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
!python model_main_tf2.py --model_dir=F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn --pipeline_config_path=F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config

In [ ]:
!python .\exporter_main_v2.py --input_type image_tensor --pipeline_config_path F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config --trained_checkpoint_dir F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn --output_directory F:/tensorflow/training_demo/exported_models/my_model

In [ ]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = r"F:\tensorflow\training_demo\images\test\DJI_0312_JPG.rf.08f23849d327cc10139c06bae49506f4.jpg"
IMAGE_PATHS1 = r"F:\tensorflow\training_demo\images\test\DJI_0312_JPG.rf.544df0e3a7b2778052c9110942b032a7.jpg"
IMAGE_PATHS2 = r"F:\tensorflow\training_demo\images\test\DJI_0316_JPG.rf.8d0013f3dfc42181047ac5be6a3eba0e.jpg"
IMAGE_PATHS3 = r"F:\tensorflow\training_demo\images\valid\DJI_0259_JPG.rf.df5eb78cf9d9c1b930a87f55c01df623.jpg"
IMAGE_PATHS4 = r"F:\tensorflow\training_demo\images\valid\DJI_0272_JPG.rf.44bf79d219441da434dd3a60f1e56d1c.jpg"

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = r'F:\tensorflow\training_demo\exported_models\my_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = r"F:\tensorflow\training_demo\annotations\label_map.pbtxt"

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))




print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS1)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')
# DISPLAYS OUTPUT IMAGE
cv2.imshow('image', image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED
cv2.waitKey(0)
cv2.destroyAllWindows()

Loading model...Done! Took 12.070135831832886 seconds
Running inference for F:\tensorflow\training_demo\images\test\DJI_0312_JPG.rf.08f23849d327cc10139c06bae49506f4.jpg... Done


In [ ]:
python model_main_tf2.py --model_dir=F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn --pipeline_config_path=F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn/pipeline.config --checkpoint_dir=F:/tensorflow/training_demo/models/my_ssd_resnet101_v1_fpn